In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import pandas as pd
import datetime
from datetime import datetime
from datetime import timedelta
from datetime import date
import numpy as np
import glob
import xz.open
from decimal import Decimal
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
#read data
agg_login_days = pd.read_csv("../output/g1jp_user_agg_login_days.csv")
login_days_inframe = pd.read_csv("../output/g1jp_user_login_days_inframe.csv")
consecutive_login_days = pd.read_csv("../output/g1jp_user_consecutive_login_days.csv")


normal_item_buy_times = pd.read_csv("../output/g1jp_user_shoptrade_item_buy_frequency.csv")
gacha_buy_times = pd.read_csv("../output/g1jp_user_gacha_buy_times.csv")
card_fortune_store_buy_times = pd.read_csv("../output/g1jp_user_card_fortune_store_buy_times.csv")
male_outfit_fortune_store_buy_times = pd.read_csv("../output/g1jp_user_male_outfit_fortune_store_buy_times.csv")
female_outfit_fortune_store_buy_times = pd.read_csv("../output/g1jp_user_female_outfit_fortune_store_buy_times.csv")

arena = pd.read_csv("../output/recommend_arena.csv")
clan_number = pd.read_csv("../output/recommend_clan_member.csv")
equipment_cards_ratio = pd.read_csv("../output/recommend_equipment_cards_ratio.csv")
fight_maxpower = pd.read_csv("../output/recommend_fight_maxpower.csv")
fight_team = pd.read_csv("../output/recommend_fight_team.csv")
#money = pd.read_csv("../output/recommend_money.csv")
monthly_card_member_growth_fund = pd.read_csv("../output/recommend_monthly_card_member_growth_fund.csv")
recharge_time = pd.read_csv("../output/recommend_recharge_time.csv")
#smurf_maxlevel = pd.read_csv("../output/recommend_userid_smurf_maxlevel.csv")



In [4]:
#main role list
mainrole = pd.read_csv('../output/recommend_maintable.csv')

arena = arena[(arena['userid'].isin(mainrole.userid)) & (arena['roleid'].isin(mainrole.roleid))]
clan_number = clan_number[(clan_number['userid'].isin(mainrole.userid)) & (clan_number['roleid'].isin(mainrole.roleid))]
equipment_cards_ratio = equipment_cards_ratio[(equipment_cards_ratio['userid'].isin(mainrole.userid)) & (equipment_cards_ratio['roleid'].isin(mainrole.roleid))]
fight_maxpower = fight_maxpower[(fight_maxpower['userid'].isin(mainrole.userid)) & (fight_maxpower['roleid'].isin(mainrole.roleid))]
fight_team = fight_team[(fight_team['userid'].isin(mainrole.userid)) & (fight_team['roleid'].isin(mainrole.roleid))]
monthly_card_member_growth_fund = monthly_card_member_growth_fund[(monthly_card_member_growth_fund['userid'].isin(mainrole.userid))]
recharge_time = recharge_time[(recharge_time['userid'].isin(mainrole.userid))]


In [5]:
#no roleid
cumulative_payment = pd.read_csv("../output/g1jp_user_cumulative_payment.csv")
cumulative_payment_inframe = pd.read_csv("../output/g1jp_user_cumulative_payment_inframe.csv")
payment_frequency = pd.read_csv("../output/g1jp_user_payment_frequency.csv")


In [6]:
A = pd.merge(agg_login_days, login_days_inframe, how = 'outer', left_on = ["userid","roleid"], right_on = ["userid","roleid"])
B = pd.merge(A, consecutive_login_days, how = 'outer', left_on = ["userid","roleid"], right_on = ["userid","roleid"])
C = pd.merge(B, clan_number, how = 'outer', left_on = ["userid","roleid"], right_on = ["userid","roleid"])
D = pd.merge(C, equipment_cards_ratio, how = 'outer', left_on = ["userid","roleid"], right_on = ["userid","roleid"])
E = pd.merge(D, fight_maxpower, how = 'outer', left_on = ["userid","roleid"], right_on = ["userid","roleid"])  #<
F = pd.merge(E, fight_team,  how = 'outer', left_on = ["userid","roleid"], right_on = ["userid","roleid"])
G = pd.merge(F, normal_item_buy_times, how = 'outer', left_on = ["userid","roleid"], right_on = ["userid","roleid"])  #<
H = pd.merge(G, gacha_buy_times, how = 'outer', left_on = ["userid","roleid"], right_on = ["userid","roleid"])
I = pd.merge(H, card_fortune_store_buy_times, how = 'outer', left_on = ["userid","roleid"], right_on = ["userid","roleid"])
J = pd.merge(I, male_outfit_fortune_store_buy_times, how = 'outer', left_on = ["userid","roleid"], right_on = ["userid","roleid"])
K = pd.merge(J, female_outfit_fortune_store_buy_times, how = 'outer', left_on = ["userid","roleid"], right_on = ["userid","roleid"])
L = pd.merge(K, arena, how = 'outer', left_on = ["userid","roleid"], right_on = ["userid","roleid"])

#the below data is from orders, so there is only userid 
#I = pd.merge(H, recharge_time, how = 'outer', left_on = ["userid","roleid"], right_on = ["userid","roleid"])
#H = pd.merge(G, monthly_card_member_growth_fund, how = 'outer', left_on = ["userid","roleid"], right_on = ["userid","roleid"])
#L = pd.merge(K, cumulative_payment, how = 'outer', on = ["userid","userid"])
#M = pd.merge(L, cumulative_payment_inframe, how = 'outer', on = ["userid","userid"])
#N = pd.merge(M , payment_frequency, how = 'outer', on = ["userid","userid"] )


In [7]:
#paid user info
# X # paid_user_info = cumulative_payment = pd.read_csv("../output/g1jp_user_cumulative_payment.csv")
paid_user_info = pd.read_csv("../output/g1jp_user_cumulative_payment.csv")

#paid_user_info = paid_user_info[(paid_user_info['cumulative_payment'] > 0) & (paid_user_info['cumulative_payment'] < 30000)]
paid_user_info.shape

(18358, 2)

In [8]:
ensemble = L

#select paid user, dicard free user
ensemble = ensemble[ensemble['userid'].isin(paid_user_info['userid'])]
ensemble = ensemble[ensemble['userid'].isin(login_days_inframe['userid'])]
ensemble = ensemble.fillna(0)
ensemble = ensemble.drop_duplicates()
ensemble.shape

(1639, 31)

In [9]:
xga_orders = pd.read_csv("/home/xl_daily_update/raw_data/cpf/xga_orders.csv", low_memory=False)

# X loc # g1jp_orders = (xga_orders.loc[xga_orders['app_id'] == 1003])[['gid','price','regdate']]
g1jp_orders = xga_orders[xga_orders['app_id'] == 1003][['gid','price','regdate']]

g1jp_orders['date'] = pd.to_datetime(g1jp_orders['regdate']).dt.date

g1jp_orders = g1jp_orders.rename(columns={'gid':'userid'})





In [10]:
start_date = pd.to_datetime(date.today() - timedelta(days = 30))
end_date = pd.to_datetime(date.today())
# X loc # payment_in30day = g1jp_orders.loc[g1jp_orders['date'].between(start_date, end_date, inclusive = True)]
payment_in30day = g1jp_orders[g1jp_orders['date'].between(start_date, end_date, inclusive = True)]

cumulative_payment_in30day = pd.DataFrame(payment_in30day.groupby('userid')['price'].sum()).rename(columns = {'price' : 'cumulative_payment'})

cumulative_payment_in30day

,cumulative_payment
userid,
10089467,389.0
10679078,429.0
11047060,1833.0
11169475,33.0
11179927,33.0
...,...
9184254,462.0
9187946,66.0
9283430,33.0


In [11]:
#establish the reference of userid and cumulative payment gorup
conditions = [
    (cumulative_payment_in30day['cumulative_payment'].ge(1) & cumulative_payment_in30day['cumulative_payment'].lt(500)),
    (cumulative_payment_in30day['cumulative_payment'].ge(500) & cumulative_payment_in30day['cumulative_payment'].lt(2000)),
    (cumulative_payment_in30day['cumulative_payment'].ge(2000) & cumulative_payment_in30day['cumulative_payment'].lt(5000)),
    (cumulative_payment_in30day['cumulative_payment'].ge(5000) & cumulative_payment_in30day['cumulative_payment'].lt(15000)),
    (cumulative_payment_in30day['cumulative_payment'].ge(15000) & cumulative_payment_in30day['cumulative_payment'].lt(30000)),
    (cumulative_payment_in30day['cumulative_payment'].ge(30000) & cumulative_payment_in30day['cumulative_payment'].lt(60000)),
    (cumulative_payment_in30day['cumulative_payment'].ge(60000))
]

group = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7']

cumulative_payment_in30day['payment_group'] = np.select(conditions, group)
cumulative_payment_in30day = cumulative_payment_in30day.reset_index()



In [12]:
cumulative_payment_in30day.groupby('payment_group')['userid'].nunique()
# cumulative_payment_in30day.groupby('payment_group').size()

payment_group
V1    473
V2    160
V3    102
V4    104
V5     74
V6     45
V7     13
Name: userid, dtype: int64

In [13]:
ensemble['userid'].astype(str)+ensemble['roleid'].astype(str)

2       3922331135500002
5        487906533500003
6        578151933500002
8        720288733500672
9        826942033500054
              ...       
4473    1399895822510811
4482    1400079216510398
4487    1400166022510822
4505    1400568122510836
4508     140060909008649
Length: 1639, dtype: object

In [14]:
cumulative_payment_in30day['userid'] = cumulative_payment_in30day['userid'].astype('int64')
cumulative_payment_in30day.dtypes

userid                  int64
cumulative_payment    float64
payment_group          object
dtype: object

In [15]:
#concatentation userid and roleid to create unique value for each characotr belong to each user
ensemble['userrole'] = ensemble.userid.map(str) + ensemble.roleid.map(str)
# ensemble['userrole'] = ensemble['userid'].astype(str)+ensemble['roleid'].astype(str)

ensemble_payment_reference = pd.merge(ensemble, cumulative_payment_in30day, how = 'left', on = ['userid', 'userid'])
(ensemble_payment_reference[['userrole', 'payment_group']]).to_csv("../output/g1jp_userrole_payment_reference.csv", index = False)

ensemble_unique = ensemble.drop(['userid','roleid'], axis = 1)

In [16]:
cos_simi_result = pd.DataFrame(cosine_similarity(ensemble_unique))
cos_simi_result

,0,1,2,3,4,5,6,7,8,9,...,1629,1630,1631,1632,1633,1634,1635,1636,1637,1638
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1634,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1635,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1636,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1637,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [17]:
userrole = (ensemble_unique.reset_index())[['userrole']]

cos_simi_userrole = pd.concat([userrole, cos_simi_result], ignore_index=False , axis = 1)
cos_simi_userrole = cos_simi_userrole.set_index('userrole')
cos_simi_userrole.columns = cos_simi_userrole.index
cos_simi_userrole

userrole,3922331135500002,487906533500003,578151933500002,720288733500672,826942033500054,8343462125500174,8343855135500544,8345607155500632,834656232505931,8346632103000421,...,139760939008599,1397758632508339,139878789008618,1399134710081,1399742116510396,1399895822510811,1400079216510398,1400166022510822,1400568122510836,140060909008649
userrole,,,,,,,,,,,,,,,,,,,,,
3922331135500002,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
487906533500003,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
578151933500002,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
720288733500672,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
826942033500054,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399895822510811,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1400079216510398,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1400166022510822,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [18]:
final_matrix = pd.DataFrame()

for i in range(len(cos_simi_userrole.index)):
    
    temp = pd.concat([pd.DataFrame([cos_simi_userrole.columns[i]]), pd.DataFrame(cos_simi_userrole.iloc[i].loc[lambda x : (x > 0.9) & (x < 1.0)].index)], axis = 1)
    temp = temp.rename(columns = { 0 : "recommend_user", "userrole" : "similar"})
    temp['recommend_user'] = temp['recommend_user'].replace(np.nan, cos_simi_userrole.columns[i])
    
    final_matrix = pd.concat([final_matrix, temp], axis = 0)

    
final_matrix

#pd.DataFrame(cos_simi_rule_index.index[cos_simi_rule_index.iloc[2] >= 0.9]).assign(target = cos_simi_rule_index.columns[2])
#pd.concat([cos_simi_rule_index.columns[2], pd.DataFrame(cos_simi_rule_index.index[cos_simi_rule_index.iloc[2] >= 0.9])], axis = 1)

,recommend_user,similar
0,3922331135500002,84714325558
1,3922331135500002,85115821576
2,3922331135500002,863313372
3,3922331135500002,11047060873
4,3922331135500002,11047060873
...,...,...
23,140060909008649,130807876562
24,140060909008649,131491238290
25,140060909008649,131972174010
26,140060909008649,133855239416


In [19]:
final_matrix = final_matrix.dropna(how = 'all')
final_matrix

,recommend_user,similar
0,3922331135500002,84714325558
1,3922331135500002,85115821576
2,3922331135500002,863313372
3,3922331135500002,11047060873
4,3922331135500002,11047060873
...,...,...
23,140060909008649,130807876562
24,140060909008649,131491238290
25,140060909008649,131972174010
26,140060909008649,133855239416


In [20]:
final_matrix.to_csv("../output/recommended_result.csv", index = False)